In [2]:
# Run this cell and select the kaggle.json file downloaded
# from the Kaggle account settings page.
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"anish1207","key":"30f6508e5154a91b43fd94ccb36ff5c1"}'}

In [3]:
!ls -lha kaggle.json

-rw-r--r-- 1 root root 65 Dec  6 11:15 kaggle.json


In [0]:
# Next, install the Kaggle API client.
!pip install -q kaggle

In [0]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [7]:
# List available datasets.
!kaggle datasets list

ref                                                           title                                                size  lastUpdated          downloadCount  
------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  
kaggle/kaggle-survey-2018                                     2018 Kaggle ML & DS Survey Challenge                  4MB  2018-11-03 22:35:07           5988  
mehdidag/black-friday                                         Black Friday                                          5MB  2018-07-25 20:49:48          16319  
spscientist/students-performance-in-exams                     Students Performance in Exams                         8KB  2018-11-09 18:25:25           1031  
szamil/who-suicide-statistics                                 WHO Suicide Statistics                              307KB  2018-08-29 21:23:11           3085  
iarunava/happy-house-dataset                        

In [8]:
!kaggle datasets download -d uciml/sms-spam-collection-dataset

  0% 0.00/208k [00:00<?, ?B/s]
100% 208k/208k [00:00<00:00, 57.4MB/s]


In [11]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../content"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

kaggle.json
sample_data
sms-spam-collection-dataset.zip



In [13]:
!unzip ../content/sms-spam-collection-dataset.zip


Archive:  ../content/sms-spam-collection-dataset.zip
  inflating: spam.csv                


In [14]:
!ls ../content

kaggle.json  sample_data  sms-spam-collection-dataset.zip  spam.csv


In [15]:
data = pd.read_csv('../content/spam.csv',encoding='latin1')
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [0]:
del data['Unnamed: 2']
del data['Unnamed: 3']
del data['Unnamed: 4']

In [17]:
data.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [18]:
data['v1'] = data['v1'].replace(['ham','spam'],[0,1])
data.head()


,v1,v2
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [19]:
y = data['v1'].as_matrix()
X_text = data['v2'].as_matrix() 
print(X_text.shape)
print(y.shape)

(5572,)
(5572,)


In [24]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [25]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
sw = stopwords.words("english")
cv = CountVectorizer(stop_words =sw)
tcv = cv.fit_transform(X_text).toarray()
#print(cv.vocabulary_)
print(len(tcv[0,:]))

8536


In [26]:
print(tcv.shape)
print(y.shape)

(5572, 8536)
(5572,)


In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words=sw,lowercase=True)
X = vectorizer.fit_transform(X_text).toarray()
print(X.shape)
print(y.shape)

(5572, 8536)
(5572,)


In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.202, random_state=42)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
y_test.shape

(4446, 8536)
(4446,)
(1126, 8536)


(1126,)

In [30]:
n_classes = 2
y_n_train = np.zeros((y_train.size,n_classes)).astype(int)
print(y_n_train.shape)
k = 0
for i in y_train:
    y_n_train[k,i] = 1
    k+=1
print(y_n_train)

(4446, 2)
[[1 0]
 [1 0]
 [1 0]
 ...
 [1 0]
 [1 0]
 [1 0]]


In [36]:
import tensorflow as tf
from tensorflow.contrib import rnn
tf.reset_default_graph()

epochs = 25
n_classes = 2
batch_size = 78
chunk_size = 97
n_chunks = 88
rnn_size = 78
x = tf.placeholder('float',[None,n_chunks,chunk_size])
y = tf.placeholder('float')

#from tensorflow.python.ops import rnn, rnn_cell
def recurrent_neural_network(x):
    layer = {'weights':tf.Variable(tf.random_normal([rnn_size,n_classes])),
             'biases':tf.Variable(tf.random_normal([n_classes]))}

    x = tf.unstack(x, n_chunks, 1)
    #print x.shape
    lstm_cell = rnn.BasicLSTMCell(rnn_size, forget_bias=1.0)

    # Get lstm cell output
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    output = tf.matmul(outputs[-1],layer['weights']) + layer['biases']

    return output
  
  
def train_neural_network(x):
    prediction = recurrent_neural_network(x)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y) )
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        for epoch in range(epochs):
            epoch_loss = 0
            i = 0
            while i<len(X_train):
                start = i
                end = i + batch_size
                #print(start,end)
                epoch_x, epoch_y = X_train[start:end],y_n_train[start:end]
                epoch_x = epoch_x.reshape((batch_size,n_chunks,chunk_size))
                #print epoch_x.shape
                _, c = sess.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y})
                epoch_loss += c
                i += batch_size

            print('Epoch', epoch, 'completed out of',epochs,'loss:',epoch_loss)

        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))

        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:',accuracy.eval({x:X_train.reshape((-1, n_chunks, chunk_size)), y:y_n_train}))
        #pred = sess.run(prediction,feed_dict={x:X_test.reshape((-1, n_chunks, chunk_size))})
        #corr = tf.argmax(pred,1)
        #corr = sess.run(corr)
        #print(corr)
        #k = [i+1 for i in range(len(corr))]
        #yg = pd.DataFrame({'ImageId':pd.Series(k),'Label':pd.Series(corr)})
        #yg.to_csv('ans.csv',index=False)
        
        
train_neural_network(x)

Epoch 0 completed out of 25 loss: 33.59570696949959
Epoch 1 completed out of 25 loss: 21.27716615796089
Epoch 2 completed out of 25 loss: 20.195585638284683
Epoch 3 completed out of 25 loss: 17.165591791272163
Epoch 4 completed out of 25 loss: 13.206546068191528
Epoch 5 completed out of 25 loss: 10.82458896189928
Epoch 6 completed out of 25 loss: 10.136157862842083
Epoch 7 completed out of 25 loss: 9.508443489670753
Epoch 8 completed out of 25 loss: 9.673760540783405
Epoch 9 completed out of 25 loss: 9.317856539040804
Epoch 10 completed out of 25 loss: 8.303602870553732
Epoch 11 completed out of 25 loss: 7.984822867438197
Epoch 12 completed out of 25 loss: 7.203532326966524
Epoch 13 completed out of 25 loss: 7.525752533227205
Epoch 14 completed out of 25 loss: 7.030496772378683
Epoch 15 completed out of 25 loss: 6.3224430195987225
Epoch 16 completed out of 25 loss: 5.660414416342974
Epoch 17 completed out of 25 loss: 5.269449204206467
Epoch 18 completed out of 25 loss: 5.47486648149788